In [19]:
from tensorflow.keras.optimizers import Adam
import numpy as np
from support import plot_reconstruction, plot_mesh
import pyeit.mesh as mesh
from pyeit.mesh.shape import thorax

In [16]:
l_path = "../data_thorax/h0=0.05_n_el=16_r_min=0.1_r_max=0.4_dist_exc=8_step_meas=4noise=Truendiv=2/"

In [17]:
tmp = np.load(l_path + "sample_000000.npz", allow_pickle=True)
tmp.files

['anomaly',
 'perm_array',
 'n_el',
 'h0',
 'v_empty',
 'v_obj',
 'dist_exc',
 'step_meas']

# Load Data

In [ ]:
# create empty mesh:
mesh_obj = mesh.create(tmp["n_el"], h0=tmp["h0"], fd=thorax)
mesh_obj.perm = tmp["perm_array"]
plot_mesh(mesh_obj)

In [ ]:
V = np.load("../data_thorax/V.npy", allow_pickle=True)
perms = np.load("../data_thorax/perms.npy", allow_pickle=True)
radii = np.load("../data_thorax/radii.npy", allow_pickle=True)

# Load VAE

In [23]:
from VAE_thorax import vae

vae.build(input_shape=())
vae.load_weights("../data_thorax/vae_w.h5")

In [ ]:
# test VAE
tt_spl = 4_000
perms_train = perms[:tt_spl]
_, _, z0 = vae.encoder.predict(np.expand_dims(perms_train[0, :], 0))
_, _, z50 = vae.encoder.predict(np.expand_dims(perms_train[50, :], 0))

plot_reconstruction(mesh_obj, perms_train[0, :], title="true")
plot_reconstruction(mesh_obj, np.concatenate(vae.decoder.predict(z0)))
plot_reconstruction(mesh_obj, perms_train[50, :], title="true")
plot_reconstruction(mesh_obj, np.concatenate(vae.decoder.predict(z50)))

# Load different Mappers